In [1]:
import requests
import bs4 as bs
import time
import lxml
import pickle
import numpy as np

In [7]:
def get_proxys():
    headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Mobile Safari/537.36'}
    fp = open('host.txt', 'w')

    for j in range(2, 7):
        print(j)
        url = 'http://www.xicidaili.com/nn/{}'.format(j)
        s = requests.get(url, headers = headers)
        soup = bs.BeautifulSoup(s.text,'lxml')
        ips = soup.select('#ip_list tr')

        for i in ips:
            try:
                ipp = i.select('td')
                ip = ipp[1].text
                host = ipp[2].text
                fp.write(ip + '\t' + host + '\n')
            except Exception as e :
                print ('no ip !')

    fp.close()


def verify_proxy():
    url = 'https://www.google.com'
    fp = open('host.txt', 'r')
    ips = fp.readlines()
    proxys = list()

    for p in ips:
        ip = p.strip('\n').split('\t')
        proxy = 'http:\\' + ip[0] + ':' + ip[1]
        proxies = {'proxy': proxy}

        try:
            s = requests.get(url, proxies=proxies)
            proxys.append(proxies)
            print(s)
        except Exception as e:
            print(e)
            
    return proxys


if __name__ == "__main__":
    get_proxys()
    proxys = verify_proxy()
    with open("proxys.pickle", "wb") as p:
        pickle.dump(proxys, p)

In [4]:
class Scraper:
    
    def __init__(self, startASIN):

        self.proxies = []
        self.ASINS = set()
        self.ASINS_now = set()
        self.ASINS_Used = set()
        self.startASIN = startASIN
        self.state = 0
        
    def getASINS(self, ASIN, proxy, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
):
        base_url = "http://www.amazon.com/dp/"
        target_url = base_url + ASIN
        try:
            print("Now processing: {}".format(ASIN))
            response = requests.get(target_url, headers=headers, proxies=proxy)
            page = bs.BeautifulSoup(response.content, "lxml")
    
            items = page.find_all("li", {"class":"a-carousel-card"})
            for item in items:
                div = item.find("div", {"class":"a-section sp_offerVertical p13n-asin"})
                if(div != None):
                    self.ASINS.add(div.attrs["data-asin"])
            
        except Exception as e:
            print(e)
            
            
    def start(self, terminated_round = 3, proxies=True):
        
        proxies_file = open("proxys.pickle", "rb")
        proxies = pickle.load(proxies_file)
                           
        print("Starting scraping ASINS!\n")
       
        if(self.state == 0):
            self.getASINS(self.startASIN, proxy = np.random.choice(proxies))
            self.ASINS_now = self.ASINS.copy()
            self.state += 1
        
        while(self.state <= terminated_round):
            print("Round: {}".format(self.state))
            print("Number of ASINS: {}\n".format(len(self.ASINS)))
            
            for ASIN in self.ASINS_now:
                if(ASIN not in self.ASINS_Used):
                    self.getASINS(ASIN, proxy = np.random.choice(proxies)) # Update self.ASINS
                    self.ASINS_Used.add(ASIN) # Update self.ASINS_used
                    self.ASINS.remove(ASIN) # In case of prevent from duplicatively iterating ASINS
                    
            self.ASINS_now = self.ASINS.copy()
            self.state += 1
    
    def writeIntoFile(self, output_Path):
        file = open(output_Path, "w")
        for ASIN in self.ASINS_Used:
            file.write(ASIN + '\n')
        
        file.close()  
        print("Finished!")

In [5]:
if __name__ == "__main__":
    
    startASIN = "B00EEZF426"
    output_Path = "ASINS.txt"
    
    sc = Scraper(startASIN)
    sc.start()
    sc.writeIntoFile(output_Path)  